<a href="https://colab.research.google.com/github/chizhang23/sentence-transformers/blob/master/tensorflow2_ranking_bert_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
cd drive/My\ Drive/Colab\ Notebooks/bert_repo

/content/drive/My Drive/Colab Notebooks/bert_repo


In [0]:
import sys


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import codecs
import collections
import json
import re
import os
import pprint
import numpy as np
import tensorflow as tf
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization
import tensorflow_hub as hub

In [0]:
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.
  # import sys

# !test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
# if not 'bert_repo' in sys.path:
#   sys.path += ['bert_repo']

# import python modules defined by BERT

TPU address is grpc://10.15.129.130:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 11450788454731632805),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 1090137685359456623),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3211553020762287545),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13131192420430595006),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 10455150058689746099),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
TASK = 'Ranking'
TASK_DATA_DIR ='/content/drive/My Drive/Colab Notebooks/'
BUCKET = 'ranking_model'
assert TASK
assert BUCKET
OUTPUT_DIR = 'gs://{}/bert-tfhub/models_all_20/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
BERT_MODEL = 'uncased_L-12_H-768_A-12' 
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [0]:

TRAIN_BATCH_SIZE = 32
# EVAL_BATCH_SIZE = 2870
EVAL_BATCH_SIZE = 1
PREDICT_BATCH_SIZE = 2816
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 20.0 
# previous was 20 ephoch
MAX_SEQ_LENGTH = 300
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 100
SAVE_SUMMARY_STEPS = 200

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
  "ranking":run_classifier.RankingProcessor,
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()

# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
print('train_steps:',num_train_steps)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
# NUM_TPU_CORES = 8
NUM_TPU_CORES = 1
ITERATIONS_PER_LOOP = 1000

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

data_dir: /content/drive/My Drive/Colab Notebooks/bert_train_all.pkl
train_steps: 4490


In [0]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  # use_tpu=False,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  # use_tpu=False,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

INFO:tensorflow:Using config: {'_model_dir': 'gs://ranking_model/bert-tfhub/models_all_20/Ranking', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.15.129.130:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f86f9b0bba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.15.129.130:8470', '_evaluation_master': 'grpc://10.15.129.130:8470', '_is_chief': T

INFO:tensorflow:Using config: {'_model_dir': 'gs://ranking_model/bert-tfhub/models_all_20/Ranking', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.15.129.130:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f86f9b0bba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.15.129.130:8470', '_evaluation_master': 'grpc://10.15.129.130:8470', '_is_chief': T

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


In [0]:
# Train the model
import datetime
def model_train(estimator):
  print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))


In [0]:
model_train(estimator_from_tfhub)

In [0]:
import datetime
import time
def model_eval(estimator):
  # Eval the model.
  start = time.time()
  eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  end= time.time()
  print('++++++++++++++++++processing time is {}'.format((end-start)))
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  print('+++++++++++processing time per sentence is {}'.format((end-start)/2970))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))

In [0]:
model_eval(estimator_from_tfhub)

In [0]:
def model_predict(estimator):
  # Make predictions on a subset of eval examples
  prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)
  result = []

  for example, prediction in zip(prediction_examples, predictions):
    # print(float(prediction['probabilities'][1]),int(example.label))
    result.append((float(prediction['probabilities'][1]),int(example.label)))
    # print('text_a: %s\ntext_b: %s\nlabel:%s\nprediction:%s\n' % (example.text_a, example.text_b, str(example.label), prediction['probabilities']))
  def get_p_at_n_in_m(data, n, m, ind):
    pos_score = data[ind][0];
    curr = data[ind:ind+m];
    curr = sorted(curr, key = lambda x:x[0], reverse=True)
    #print(curr,n,m,ind)
    if curr[n-1][0] <= pos_score:
        return 1;
    return 0;

  def evaluate(data):
      p_at_1_in_2 = 0.0
      p_at_1_in_10 = 0.0
      p_at_2_in_10 = 0.0
      p_at_5_in_10 = 0.0
      length = 0
      a = 0
      b = 0
      while a < len(data):
          if data[a][1] == 1:
              b=a+1
              while b<len(data)-1 and data[b][1] ==0:
                  b = b+1
              length +=1
              assert data[a][1] == 1
              if (b-a)>5:
                  p_at_1_in_2 += get_p_at_n_in_m(data, 1, 2, a)
                  p_at_1_in_10 += get_p_at_n_in_m(data, 1, (b-a), a)
                  p_at_2_in_10 += get_p_at_n_in_m(data, 2, (b-a), a)
                  p_at_5_in_10 += get_p_at_n_in_m(data, 5, (b-a), a)
              a = b
          else:
              a=a+1
      return (p_at_1_in_2/length, p_at_1_in_10/length, p_at_2_in_10/length, p_at_5_in_10/length)
  print ('result_num',len(result))
  R12,R110,R210,R510 = evaluate(result)
  print('R1@2',R12,
        'R1@10',R110,
        'R2@10',R210,
        'R5@10',R510 )
    # return result

In [0]:
model_predict(estimator_from_tfhub) 

data_dir: /content/drive/My Drive/Colab Notebooks/bert_val_all.pkl
INFO:tensorflow:Writing example 0 of 2816


INFO:tensorflow:Writing example 0 of 2816


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: dev-0


...

INFO:tensorflow:Outfeed finished for iteration (0, 0)


INFO:tensorflow:Outfeed finished for iteration (0, 0)


result_num 2816
R1@2 0.9574468085106383 R1@10 0.8191489361702128 R2@10 0.9290780141843972 R5@10 0.9858156028368794
INFO:tensorflow:prediction_loop marked as finished
ERROR:tensorflow:Error recorded from infeed: Step was cancelled by an explicit call to `Session::Close()`.


INFO:tensorflow:prediction_loop marked as finished
ERROR:tensorflow:Error recorded from infeed: Step was cancelled by an explicit call to `Session::Close()`.


Exception ignored in: <generator object TPUEstimator.predict at 0x7f179a66be08>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/tpu/tpu_estimator.py", line 3078, in predict
    rendezvous.raise_errors()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/tpu/error_handling.py", line 140, in raise_errors
    six.reraise(typ, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/six.py", line 693, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/tpu/error_handling.py", line 109, in catch_errors
    yield
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_estimator/python/estimator/tpu/tpu_estimator.py", line 536, in _run_infeed
    session.run(self._enqueue_ops)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py", line 956, in run
    run_metadata_ptr)
  File "/usr/loc

In [0]:
import tensorflow as tf
tf.__version__

'1.15.0'